Only reading in one email for now

In [1]:
emailDir = "./maildir/maildir"
enroncsv = "../experiments/enron.csv"
metadataHeaders = '../experiments/metadataHeaders.csv'
spam = '../experiments/spam.csv'
spamSubjects = '../experiments/spamWords.txt'

In [2]:
import pandas as pd

columns = pd.read_csv(metadataHeaders, sep=',').columns.tolist()
columns

['Filename',
 'Person',
 'Directory',
 'Message-ID',
 'Date',
 'From',
 'To',
 'Subject',
 'Cc',
 'Time',
 'Attendees',
 'Re',
 'Mime-Version',
 'Content-Type',
 'Content-Transfer-Encoding',
 'Bcc',
 'X-From',
 'X-To',
 'X-cc',
 'X-bcc',
 'X-Folder',
 'X-Origin',
 'X-FileName']

In [3]:
df = pd.read_csv(enroncsv, names=columns, sep='|', low_memory=False)

print("-- DONE --")

-- DONE --


Lets query for a mail that does not have `Message-ID` with `.JavaMail.evans@thyme`

In [4]:
result = df[pd.notnull(df['Message-ID'])]

result = result.loc[~result['Message-ID'].str.contains('.JavaMail.evans@thyme')]
result

,Filename,Person,Directory,Message-ID,Date,From,To,Subject,Cc,Time,...,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName


In [5]:
result = df[pd.notnull(df['Subject'])]

result = result.loc[result['Subject'].str.contains('Porn')]
len(result)

25

No results so the server `.JavaMail.evans@thyme` is not helpfull

Lets count the number of non null fields per header and make it a percentage over the total number of parsable emials

In [6]:
import matplotlib.pyplot as plt

numberOfRows = len(df.index)
missingData = (numberOfRows - df.count()) / numberOfRows * 100
plot = missingData.plot(kind='bar')

plt.show()
missingData

<Figure size 640x480 with 1 Axes>

Filename                      0.000000
Person                        0.000000
Directory                     0.000000
Message-ID                    0.000387
Date                          0.000387
From                          0.000387
To                            4.222880
Subject                       3.713189
Cc                           75.279489
Time                         99.999807
Attendees                    99.994201
Re                           99.998840
Mime-Version                  0.011597
Content-Type                  0.011597
Content-Transfer-Encoding     0.010631
Bcc                          75.284322
X-From                        0.011404
X-To                          1.770680
X-cc                         75.084079
X-bcc                        99.959410
X-Folder                      0.012370
X-Origin                      0.006958
X-FileName                    0.820493
dtype: float64

How many people flolders don't match their X-origin

In [7]:
len(df.loc[df['Person'] != df['X-Origin'].str.lower()])

35496

How many different `Content-Type`'s are in the dataset

In [8]:
df['Content-Type'].unique()

array([nan, 'text/plain; charset="us-ascii"',
       'text/plain; charset="ANSI_X3.4-1968"'], dtype=object)

How many different `Content-Transfer-Encoding`'s are in the dataset

In [9]:
df['Content-Transfer-Encoding'].unique()

array([nan, '7bit', 'quoted-printable', 'base64',
       '\\JSKILLIN (Non-Privileged)\\Deleted Items', '\\jskillin\\Inbox',
       '\\HARORA (Non-Privileged)\\Arora, Harry\\Deleted Items',
       '\\HARORA (Non-Privileged)\\Arora, Harry\\Inbox',
       'EID: <24606> ERe: <0>'], dtype=object)

How many different `Mime-Version`'s are in the dataset

In [10]:
df['Mime-Version'].unique()

array([nan,  1.])

Lets remove those colums that are almost never filled in or that are irrelevent

In [11]:
removableColumns = ['Time', 'Attendees', 'Bcc', 'X-bcc', 'X-cc', 'X-Origin']

import csv
with open('removableColumns.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for col in removableColumns:
        writer.writerow([col])

df.drop(removableColumns, axis=1)

print('-- DONE --')

-- DONE --


In [12]:
listOfEmailsForBagOfWords = df.loc[df['Directory'].str.contains('inbox')]

print('-- DONE --')

-- DONE --


In [13]:
wordsPerEmail = df[pd.notnull(df['Subject'])]
nonSpamSubjects = wordsPerEmail['Subject']
wordsPerEmail = nonSpamSubjects.str.split(' ')
len(wordsPerEmail)

498161

In [14]:
import string
printable = set(string.printable)

def isEnglish(s):
    for x in s:
        if x not in printable:
            return False
    return True

def addToBagOfWords(dictionary, arr):
    for word in arr:
        word = word.lower()
        
        if '=' in word:
            continue
        if '/' in word:
            continue
        if '\\' in word:
            continue
        if '_' in word:
            continue
        if '-' in word:
            continue
        if ':' in word:
            continue
        if '@' in word:
            continue
        if '#' in word:
            continue
        if '$' in word:
            word = '$'

        word = word.replace('.', '')
        word = word.replace(')', '')
        word = word.replace('(', '')
        word = word.replace('&', '')
        word = word.replace('\'', '')
        word = word.replace('\"', '')
        word = word.replace(',', '')
        word = word.replace('[', '')
        word = word.replace(']', '')
        word = word.replace('{', '')
        word = word.replace('}', '')
        word = word.replace(';', '')

        if word == '':
            continue

        if word in dictionary:
            dictionary[word] = dictionary[word] + 1
        else:
            dictionary[word] = 1
            
    return dictionary

In [15]:
def addToDictionary(dictionary, arr):
    for index, subject in arr.iteritems():
        dictionary = addToBagOfWords(dictionary, subject)
                
    return dictionary

In [16]:
dictionary = {}

dictionary = addToDictionary(dictionary, wordsPerEmail)
        
len(dictionary)

39519

In [19]:
spamSubjects = pd.read_csv(spamSubjects, names=['Subject'], sep=',')

In [20]:
wordsPerEmail = spamSubjects['Subject'].str.split(' ')

dictionary = addToDictionary(dictionary, wordsPerEmail)
        
len(dictionary)

39549

In [21]:
dictionaryList = []
for key in dictionary:
    dictionaryList.append([key, dictionary[key]])
    
dictionaryList = pd.DataFrame(dictionaryList, columns=["Word", "Occurances"])
len(dictionaryList)

39549

In [22]:
dictionaryList = dictionaryList.loc[dictionaryList['Occurances'] > 30].reset_index(drop=True)
len(dictionaryList)

5558

In [24]:
import matplotlib.pyplot as plt
import numpy as np

training = []

NON_SPAM_SAMPLES = 1000
SPAM_SAMPLES = len(spamSubjects)

trainingPos = df[pd.notnull(df['Subject'])]
trainingPos = trainingPos.sample(NON_SPAM_SAMPLES)
trainingPos = trainingPos['Subject']

trainingPosSplit = trainingPos.str.split(' ')

for index, subject in trainingPosSplit.iteritems():
    posBagOfWords = {}
    posBagOfWords = addToBagOfWords(posBagOfWords, subject)
    
    listOfIn = []
    for key in posBagOfWords:
        listOfIn.append(key)
    
    inputBag = dictionaryList['Word'].isin(listOfIn).values.tolist()
    
    training.append([np.array(inputBag), np.array([0])])
    
trainingNeg = spamSubjects.sample(SPAM_SAMPLES)
trainingNeg = trainingNeg['Subject']

trainingNegSplit = trainingNeg.str.split(' ')

for index, subject in trainingNegSplit.iteritems():
    negBagOfWords = {}
    negBagOfWords = addToBagOfWords(negBagOfWords, subject)
    
    listOfIn = []
    for key in negBagOfWords:
        listOfIn.append(key)
    
    inputBag = dictionaryList['Word'].isin(listOfIn).values.tolist()
    
    training.append([np.array(inputBag), np.array([1])])

training = np.array(training)
np.random.shuffle(training)

len(training)

1249

In [25]:
from mlpy.dataSet.dataSetTool import DataSetTool
from mlpy.numberGenerator.bounds import Bounds
from mlpy.neuralNetwork.feedForwardNeuralNetwork import NeuralNetwork
from mlpy.neuralNetwork.structure.layer import Layer

l_rate = 0.001
bounds = Bounds(-2, 2)

inputLayer = Layer(bounds, size = len(training[0][0]), prev = None, l_rate = l_rate, bias = True, label = "Input layer")
hiddenLayer = Layer(bounds, size = 200, prev = inputLayer, l_rate = l_rate, bias = True, label = "Hidden layer")
hiddenLayer2 = Layer(bounds, size = 20, prev = hiddenLayer, l_rate = l_rate, bias = True, label = "Hidden layer 2")
outputLayer = Layer(bounds, size = len(training[0][1]), prev = hiddenLayer2, l_rate = l_rate, bias = False, label = "Output layer")

fnn = NeuralNetwork()
fnn.appendLayer(inputLayer)
fnn.appendLayer(hiddenLayer)
fnn.appendLayer(hiddenLayer2)
fnn.appendLayer(outputLayer)

group_training = np.array([input[0] for input in training])
group_target = np.array([output[1] for output in training])

errors = []

In [26]:
print("Starting..")
for i in range(4000):
    mod = i % len(training)
    in_out = training[mod]
    result = fnn.fire(group_training)
    i_error = fnn.backPropagation(group_target)
    
    if i % 100 == 0:
        print(i_error.mean())
    
print("-- DONE --")

Starting..
-0.36775262841674716
0.04877275053931843
0.02133254372429527
0.008372047465422526
0.002916901445840433
0.00035713937182961275
-0.0012791167002708917
-0.0024987526170419188
-0.0034607732719814147
-0.004230932293276721
-0.004856486333782484
-0.005393664186084141
-0.005879687916016475
-0.00631501683153125
-0.006688113915487953
-0.006990866677513786
-0.0072192457773339845
-0.007366065528199428
-0.0074066842124670285
-0.007305342144923007
-0.007059747409307591
-0.006693084378155808
-0.0062453237638714605
-0.005776479131806317
-0.005340142985604143
-0.004967821507346416
-0.004680312458473453
-0.004501514529929714
-0.004452367417277723
-0.004500285291124079
-0.00456463273920163
-0.004497407311918768
-0.004041116702268134
-0.003395699095019259
-0.0028247333007924568
-0.0023574413873951507
-0.001939945941744941
-0.0015781632517008705
-0.0012959629984396212
-0.0010807934288644308
-0.0008881385626342962
-0.0006765000556792661
-0.0004379277544421446
-0.0001749885803291419
0.000148461767

In [32]:
testing = []

TEST_NON_SPAM_SAMPLES = 250
TEST_SPAM_SAMPLES = len(spamSubjects)

trainingPos = df[pd.notnull(df['Subject'])]
trainingPos = trainingPos.sample(TEST_NON_SPAM_SAMPLES)
trainingPos = trainingPos['Subject']

trainingPosSplit = trainingPos.str.split(' ')

for index, subject in trainingPosSplit.iteritems():
    posBagOfWords = {}
    posBagOfWords = addToBagOfWords(posBagOfWords, subject)
    
    listOfIn = []
    for key in posBagOfWords:
        listOfIn.append(key)
    
    inputBag = dictionaryList['Word'].isin(listOfIn).values.tolist()
    
    testing.append([np.array(inputBag), np.array([0])])
    
trainingNeg = spamSubjects.sample(TEST_SPAM_SAMPLES)
trainingNeg = trainingNeg['Subject']

trainingNegSplit = trainingNeg.str.split(' ')

for index, subject in trainingNegSplit.iteritems():
    negBagOfWords = {}
    negBagOfWords = addToBagOfWords(negBagOfWords, subject)
    
    listOfIn = []
    for key in negBagOfWords:
        listOfIn.append(key)
    
    inputBag = dictionaryList['Word'].isin(listOfIn).values.tolist()
    
    testing.append([np.array(inputBag), np.array([1])])

testing = np.array(testing)
np.random.shuffle(testing)

len(testing)

499

In [33]:
correct = 0
spamCorrect = 0
nonSpamCorrect = 0
for i in range(len(testing)):
    in_out = testing[i]
    result = fnn.fire(np.array([in_out[0]]))
    
    target = in_out[1][0]
    result = np.round(result[0][0])

    if result == target:
        correct += 1
        if target == 1:
            spamCorrect += 1
        else:
            nonSpamCorrect += 1
        
        
print("classification accuracy: ", correct / len(testing))
print("Non spam classification accuracy: ", nonSpamCorrect / TEST_NON_SPAM_SAMPLES)
print("Spam classification accuracy: ", spamCorrect / TEST_SPAM_SAMPLES)

classification accuracy:  0.9018036072144289
Non spam classification accuracy:  0.916
Spam classification accuracy:  0.8875502008032129
